In [ ]:
import numpy as np
all_features_fc2 = np.load('/content/drive/MyDrive/VGG_features/ver.2/all_features_fc2.npy')

In [ ]:
import keras
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model, Sequential 
from keras.layers import AveragePooling2D, Dense, Dropout, Flatten, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

In [ ]:
vgg16_model = VGG16(include_top = True, input_shape=(224, 224, 3))

model = Sequential()
# model.summary()

for layer in vgg16_model.layers[:-1]:
    model.add(layer)
# model.summary()

for layer in model.layers:
    layer.trainable = False

# model.add(Dense(256, activation='relu'))
# model.add(Dense(128, activation='relu'))
# model.add(Dense(64, activation='relu')) #
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       2

In [ ]:
y_train = pd.read_csv('y_train.csv')
y_valid = pd.read_csv('y_valid.csv')
y_test = pd.read_csv('y_test.csv')

In [ ]:
from sklearn.metrics import mean_squared_error
def rmse(y_test, y_predict):
  return np.sqrt(mean_squared_error(y_test,y_predict))

## Dimension Reduction by Adding Dense layers

In [ ]:
# get the best param
all_params = []
# print(x_train[0], y_train[0], x_valid[0], y_valid[0])
for d1 in [4096, 2048, 1024, 512, 256, 128, 64]:
  for d2 in [4096, 2048, 1024, 512, 256, 128, 64]:
    for d3 in [64, 32, 16, 8]:
      model1 = Sequential()
      model1.add(Dense(d1, activation='PReLU'))
      model1.add(Dropout(0.5))
      model1.add(Dense(d2, activation='PReLU'))
      model1.add(Dropout(0.5))
      model1.add(Dense(d3))
      features_loop_reg = model1.predict(all_features_fc2)
      print('d1:', d1)
      print('d2:', d2)
      print('d3:', d3)
      ##############################################################################
      # RANDOM FOREST
      # validation
      from sklearn.ensemble import RandomForestRegressor
      RF_reg_model = RandomForestRegressor(n_estimators=20, random_state=42)
      x_train = features_loop_reg[:10049]
      y_train = y_train_price
      RF_reg_model.fit(x_train, y_train)

      x_valid = features_loop_reg[10049:11149]
      y_valid = y_valid_price
      y_reg_pred = RF_reg_model.predict(x_valid)
      rmse_val = rmse(y_valid, y_reg_pred)
      print('RMSE:', rmse_val)
      all_params.append((d1, d2, d3, rmse))
      print('--------------------------------------')

d1: 4096
d2: 4096
d3: 64
RMSE: 51589.06065427314
--------------------------------------
d1: 4096
d2: 4096
d3: 32
RMSE: 51334.15031652145
--------------------------------------
d1: 4096
d2: 4096
d3: 16
RMSE: 52627.765605914305
--------------------------------------
d1: 4096
d2: 4096
d3: 8
RMSE: 53506.07043816414
--------------------------------------
d1: 4096
d2: 2048
d3: 64
RMSE: 50956.81548089622
--------------------------------------
d1: 4096
d2: 2048
d3: 32
RMSE: 51633.830734133255
--------------------------------------
d1: 4096
d2: 2048
d3: 16
RMSE: 51915.8922935666
--------------------------------------
d1: 4096
d2: 2048
d3: 8
RMSE: 54860.00825109487
--------------------------------------
d1: 4096
d2: 1024
d3: 64
RMSE: 49829.32815807187
--------------------------------------
d1: 4096
d2: 1024
d3: 32
RMSE: 50659.61048163519
--------------------------------------
d1: 4096
d2: 1024
d3: 16
RMSE: 53133.37541574321
--------------------------------------
d1: 4096
d2: 1024
d3: 8
RMSE: 549

#### Best Dimension Reduction
- d1: 4096
- d2: 1024
- d3: 64

In [ ]:
model2 = Sequential()
model2.add(Dense(4096, activation='PReLU'))
model2.add(Dense(1024, activation='PReLU'))
model2.add(Dense(64)) 
features_final_64 = model2.predict(all_features_fc2)

# np.save('features_64_final', features_final_64)
from numpy import savetxt
savetxt('features_64_final.csv', features_final_64, delimiter=',')

## PCA

In [ ]:
import sklearn
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import PCA

# iris = datasets.load_iris()
# Xdata = iris.data
rmse_lst = []
components = [16, 32, 48, 64]    
for n in components:
    pca = PCA(n_components=n)
    recon = pca.inverse_transform(pca.fit_transform(all_features_fc2))
    rmse = mean_squared_error(all_features_fc2[0], recon[0], squared=False)
    rmse_lst.append(rmse)
    print("RMSE: {} with {} components".format(rmse, n))

RMSE: 0.5664337873458862 with 16 components
RMSE: 0.5084164142608643 with 32 components
RMSE: 0.43904605507850647 with 48 components
RMSE: 0.4025101661682129 with 64 components


In [ ]:
from sklearn.decomposition import PCA
pca_64 = PCA(n_components=64)
reduced_features_64 = pca_64.fit_transform(all_features_fc2)
# print(reduced_features_8.shape)

In [ ]:
from numpy import savetxt
savetxt('pca_features_64.csv', reduced_features_64, delimiter=',')